<a href="https://colab.research.google.com/github/pmj-chosim/Commit-Project-2023.1.20-2023.2.28-/blob/main/2023.02.08/CV/4_transfertrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터셋 불러오기 - #1

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
import torchvision.models as models
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [ ]:
# 데이터셋 불러오기
transform_train = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize((256, 256)),
                                      transforms.CenterCrop(224),
                                      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([transforms.ToTensor(),
                                     transforms.Resize((256, 256)),
                                     transforms.CenterCrop(224),
                                     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)), 
])

train_set = datasets.CIFAR10(
    root='./data',        # 데이터 저장 위치
    train=True,           # True: train set, False: test set
    download=True,       # 다운로드 여부, (이미 다운받았으면 False로 지정)
    transform=transform_train   # 데이터 선처리 작업
)

test_set = datasets.CIFAR10(
    root='./data',        # 데이터 저장 위치
    train=False,           # True: train set, False: test set
    download=True,       # 다운로드 여부, (이미 다운받았으면 False로 지정)
    transform=transform_test   # 데이터 선처리 작업
)

batch_size = 128
train_loader = DataLoader(train_set, batch_size=batch_size)
test_loader = DataLoader(test_set, batch_size=batch_size)

# gpu를 사용할 수 있으면 gpu, 아니면 cpu 사용
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


device(type='cuda', index=0)

# 모델 생성 - #2

In [ ]:
# 일반 Alexnet 생성
alexnet = models.alexnet().to(device)

# classifier 변경
num_classes = 10
num_ftrs = alexnet.classifier[6].in_features
alexnet.classifier[6] = nn.Linear(num_ftrs, num_classes).to(device)

In [ ]:
# 사전 학습된 Alexnet 생성
alexnet_finetune = models.alexnet(pretrained=True).to(device)

# classifier 변경
num_classes = 10
num_ftrs = alexnet_finetune.classifier[6].in_features
alexnet_finetune.classifier[6] = nn.Linear(num_ftrs, num_classes).to(device)

# CNN Backbone 프리징
for p in alexnet_finetune.features.parameters():
  p.requires_grad = False

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

# 학습 및 평가 - #3

In [ ]:
def train_and_valid(model, Epoch, learning_rate):
  # optimizer, loss function 설정
  optimizer = optim.SGD(model.parameters(), lr=learning_rate)
  criterion = nn.CrossEntropyLoss()

  # 모델 학습 모드
  model.train()

  # 에폭만큼 반복
  for epoch in range(Epoch):
    print(f'Epoch {epoch}:')
    train_loss = 0
    
    ###### 모델 학습 ######
    print(f'Training model...')
    # dataloader를 통한 데이터 불러오기
    for image, label in tqdm(train_loader):
      # 데이터 연산 위치 지정 (GPU or CPU)
      image = image.to(device)
      label = label.to(device)

      # 모델에 입력 (순전파)
      pred = model(image)
      
      # Loss 계산
      loss = criterion(pred, label)

      # 역전파
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # 최종 loss 누적
      train_loss += loss.detach().cpu().numpy()

    # 학습 loss 기록
    train_loss /= len(train_loader)
    writer.add_scalar("train_loss", train_loss, epoch)
    print(f'Train Loss: {train_loss}\n')

    ###### 모델 평가 ######
    print(f'Validating model...')
    accuracy = 0
    valid_loss = 0

    # 모델 평가 모드
    model.eval()

    # gradient 계산 차단
    with torch.no_grad():
      # Dataloader로 데이터 불러오기
      for image, label in tqdm(test_loader):
        # 데이터 연산 위치 지정 (GPU or CPU)
        image = image.to(device)
        label = label.to(device)

        # 모델에 입력 (순전파)
        pred = model(image)

        # Loss 계산
        loss = criterion(pred, label)
        
        # 최종 loss 누적
        valid_loss += loss.detach().cpu().numpy()

        # 정답 개수 누적
        corr = (pred.argmax(axis=1) == label).sum()
        accuracy += corr.detach().cpu().numpy()

    # 평가 loss 기록
    valid_loss /= len(test_loader)
    writer.add_scalar("valid_loss", valid_loss, epoch)
    print(f'Valid Loss: {valid_loss}')

    # 정확도 계산
    accuracy /= len(test_set)
    print(f'Accuracy: {100*accuracy}%\n')

In [ ]:
# base alexnet
writer = SummaryWriter(f'log/alexnet/base')
train_and_valid(alexnet, Epoch=5, learning_rate=0.1)

Epoch 0:
Training model...


100%|██████████| 391/391 [03:15<00:00,  2.00it/s]


Train Loss: 2.135240194437754

Validating model...


100%|██████████| 79/79 [00:27<00:00,  2.92it/s]


Valid Loss: 1.9207352973237823
Accuracy: 30.39%

Epoch 1:
Training model...


...

Valid Loss: 1.1759900677053234
Accuracy: 60.480000000000004%

Epoch 4:
Training model...


100%|██████████| 391/391 [03:07<00:00,  2.09it/s]


Train Loss: 0.9283120674855264

Validating model...


100%|██████████| 79/79 [00:31<00:00,  2.53it/s]

Valid Loss: 1.0526409149169922
Accuracy: 64.21%



In [ ]:
# pretrained alexnet
writer = SummaryWriter(f'log/alexnet/finetune')
train_and_valid(alexnet_finetune, Epoch=5, learning_rate=0.001)

Epoch 0:
Training model...


100%|██████████| 391/391 [02:40<00:00,  2.43it/s]


Train Loss: 1.2214996615029357

Validating model...


100%|██████████| 79/79 [00:26<00:00,  3.01it/s]


Valid Loss: 0.8688282551644724
Accuracy: 71.35000000000001%

Epoch 1:
Training model...


...

Valid Loss: 0.6372080550918097
Accuracy: 77.92%

Epoch 4:
Training model...


100%|██████████| 391/391 [02:42<00:00,  2.40it/s]


Train Loss: 0.6092461640267726

Validating model...


100%|██████████| 79/79 [00:28<00:00,  2.82it/s]

Valid Loss: 0.6121058777163301
Accuracy: 78.92%



In [ ]:
%load_ext tensorboard
%tensorboard --logdir log